In [1]:
# Basics -----------------------------------------------------
import pickle

import numpy as np
import pandas as pd

# Word2vec ----------------------------------------------------
import gensim

# NLTK -----------------------------------------------------

import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
#stop_words = set(nltk.corpus.stopwords.words("english"))

# Keras -----------------------------------------------

# from keras.layers import Dense
# from keras.models import Sequential
# from keras.utils import to_categorical
# from keras.callbacks import EarlyStopping


In [2]:
# pkl_file = open('/media/polichinel/DATA/backup/PDS_DATA/time_lines.pkl', 'rb') # from feature set.

# time_lines = pickle.load(pkl_file)

# pkl_file.close()

## Data

In [2]:
#df_unlabled_big = pd.read_csv("/media/polichinel/DATA/backup/PDS_DATA/new_many_tweets.csv", index_col= 0)
df_unlabled_big1 = pd.read_csv("new_all_unlabeled_tweets1.csv", index_col= 0)
df_unlabled_big2 = pd.read_csv("new_all_unlabeled_tweets2.csv", index_col= 0)

df_unlabled = pd.read_csv("all_unlabled_tweets.csv", index_col= 0)
df_labled = pd.read_csv("labled_tweets.csv", index_col= 0)

pkl_file = open('full_remove.pkl', 'rb') # from feature set.
full_remove = pickle.load(pkl_file)
pkl_file.close()

/home/polichinel/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (0,1,4,7,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Functions

In [ ]:
# # should be enough - check before deleting the direct implimentation

# pkl_file = open('prep2.pkl', 'rb') # from feature set.
# prep2 = pickle.load(pkl_file)
# pkl_file.close()

In [3]:
# Function from prerpossecing and feature selection

stop_words = []#["goptaxscam"] # for now no stopwords

def prep2(text):
    wordlist = nltk.word_tokenize(text)
    wordlist = [lemmatizer.lemmatize(w.lower()) for w in wordlist]
    wordlist = [w for w in wordlist if w not in full_remove and w not in stop_words]
    return wordlist


In [4]:
def check_model(model, most_sim = "president"):
    print("Most similar to {}:\n".format(most_sim))
    for i in model.wv.most_similar(most_sim):
        print(i)

    print("\n")
    # get the Most common words
    print("Most common words:")
    print(model.wv.index2word[0], model.wv.index2word[1], model.wv.index2word[2])

    print("\n")
    # get the least common words
    vocab_size = len(model.wv.vocab)
    print("Least common words:")
    print(model.wv.index2word[vocab_size - 1], model.wv.index2word[vocab_size - 2], model.wv.index2word[vocab_size - 3])

    print("\n")
    # some similarity fun
    print("Sanity sim check1:")
    print(model.wv.similarity('woman', 'man'), model.wv.similarity('woman', 'thing'))

    print("\n")
    print("Sanity sim check2:")
    # what doesn't fit?
    print(model.wv.doesnt_match("green blue red man".split()))

# Creating the model:

# Twitter

### Big

In [8]:
sentences1 = [prep2(sent) for sent in nltk.sent_tokenize(df_unlabled_big1.text.to_string())]
sentences2 = [prep2(sent) for sent in nltk.sent_tokenize(df_unlabled_big2.text.to_string())]
sentences = sentences1 + sentences2

In [9]:
print(len(sentences))
print(type(sentences))

195473
<class 'list'>


In [10]:
model = gensim.models.Word2Vec(sentences, iter=10, min_count=10, size=300,) # virker meget bedre med min_count = 10
len(list(model.wv.vocab.keys())) # Men, med min_count = 10 har du kun 1650 obs... Flere tweets mere text.


24208

In [11]:
check_model(model)

Most similar to president:

('pres', 0.7571897506713867)
('administration', 0.6175614595413208)
('adminis', 0.5768076777458191)
('president-elect', 0.5733792185783386)
('administ', 0.5686612129211426)
('admin', 0.565226674079895)
('administratio', 0.5596060156822205)
('administrat', 0.5422479510307312)
('presi', 0.522792637348175)
('donald', 0.5141134262084961)


Most common words:
with you today


Least common words:
opportunityagenda hcsosheriff sfareachamber


Sanity sim check1:
0.27627604539667955 0.06572580374230565


Sanity sim check2:
man


### Small

In [12]:
# tweets are back from 140 character limit; not 240!
sentences = [prep2(sent) for sent in nltk.sent_tokenize(df_unlabled.text.to_string())]
model = gensim.models.Word2Vec(sentences, iter=10, min_count=10, size=300,) # virker meget bedre med min_count = 10
len(list(model.wv.vocab.keys())) # Men, med min_count = 10 har du kun 1650 obs... Flere tweets mere text.


1650

In [13]:
check_model(model)

Most similar to president:

('trump', 0.9045068025588989)
('obama', 0.8545442819595337)
('donald', 0.84553062915802)
('admin', 0.836225152015686)
('mr.', 0.7751523852348328)
('order', 0.7572319507598877)
('decision', 0.7467522621154785)
('administration', 0.7373746633529663)
('executive', 0.735882043838501)
('comey', 0.732078492641449)


Most common words:
this with you


Least common words:
hazleton thewilsontimes ppact


Sanity sim check1:
0.6184895525381566 0.4613569059964778


Sanity sim check2:
man


In [14]:
output = open('word2vec_small_model.pkl', 'wb')
pickle.dump(model, output)
output.close()

# Reddit


In [15]:
path = "/media/polichinel/DATA/backup/PDS_DATA/"

### hot_comments_politics.pkl

In [16]:
spec_path = path + "hot_comments_politics.pkl"


pkl_file = open(spec_path, 'rb') # from feature set.
comments1 = pickle.load(pkl_file)
pkl_file.close()

# tweets are back from 140 character limit; not 240!
sentences = [prep2(sent) for sent in nltk.sent_tokenize(pd.Series(comments1).to_string())]
model = gensim.models.Word2Vec(sentences, iter=10, min_count=10, size=300,) # virker meget bedre med min_count = 10
len(list(model.wv.vocab.keys())) # Men, med min_count = 10 har du kun 1650 obs... Flere tweets mere text.

2009

In [17]:
check_model(model)

Most similar to president:

('donald', 0.920943021774292)
('twitter', 0.8230994939804077)
('team', 0.8208926320075989)
('trump', 0.8097741007804871)
('mr.', 0.808213472366333)
('asked', 0.8029544353485107)
('gowdy', 0.802731990814209)
('trey', 0.799493670463562)
('campaign', 0.7989790439605713)
('putin', 0.7971084117889404)


Most common words:
that you this


Least common words:
biden agency parson


Sanity sim check1:
0.7974816789762588 0.5014077876406039


Sanity sim check2:
man


### controversial_comments_politics.pkl

In [18]:
spec_path = path + "controversial_comments_politics.pkl"

pkl_file = open(spec_path, 'rb') # from feature set.
comments2 = pickle.load(pkl_file)
pkl_file.close()

# tweets are back from 140 character limit; not 240!
sentences = [prep2(sent) for sent in nltk.sent_tokenize(pd.Series(comments2).to_string())]
model = gensim.models.Word2Vec(sentences, iter=10, min_count=10, size=300,) # virker meget bedre med min_count = 10
len(list(model.wv.vocab.keys())) # Men, med min_count = 10 har du kun 1650 obs... Flere tweets mere text.

4397

In [19]:
check_model(model)

Most similar to president:

('biden', 0.7018259763717651)
('office', 0.6553453803062439)
('running', 0.6535567045211792)
('senate', 0.6323044300079346)
('2012', 0.6304991245269775)
('justice', 0.6250301003456116)
('female', 0.6196533441543579)
('tpp', 0.6081812381744385)
('bush', 0.5983028411865234)
('cand', 0.5941892266273499)


Most common words:
you that n't


Least common words:
h-1b infantry pyramid


Sanity sim check1:
0.4032415535639872 0.24680370158041864


Sanity sim check2:
man


### hot_comments_news.pkl

In [20]:
spec_path = path + "hot_comments_news.pkl"

pkl_file = open(spec_path, 'rb') # from feature set.
comments3 = pickle.load(pkl_file)
pkl_file.close()

# tweets are back from 140 character limit; not 240!
sentences = [prep2(sent) for sent in nltk.sent_tokenize(pd.Series(comments3).to_string())]
model = gensim.models.Word2Vec(sentences, iter=10, min_count=10, size=300,) # virker meget bedre med min_count = 10
len(list(model.wv.vocab.keys())) # Men, med min_count = 10 har du kun 1650 obs... Flere tweets mere text.

913

In [21]:
check_model(model)

Most similar to president:

('conservative', 0.9998845458030701)
('freedom', 0.9998729825019836)
('nah', 0.9998723268508911)
('asking', 0.999870777130127)
('depends', 0.999870777130127)
('charge', 0.9998691082000732)
('thanks', 0.9998683929443359)
('con', 0.9998666644096375)
('able', 0.9998666644096375)
('obama', 0.9998661279678345)


Most common words:
you that n't


Least common words:
dealer preacher bluehole


Sanity sim check1:
0.9994451437714044 0.999103754053145


Sanity sim check2:
man


### controversial_comments_news.pkl

In [22]:
spec_path = path + "controversial_comments_news.pkl"

pkl_file = open(spec_path, 'rb') # from feature set.
comments4 = pickle.load(pkl_file)
pkl_file.close()

# tweets are back from 140 character limit; not 240!
sentences = [prep2(sent) for sent in nltk.sent_tokenize(pd.Series(comments4).to_string())]
model = gensim.models.Word2Vec(sentences, iter=10, min_count=10, size=300,) # virker meget bedre med min_count = 10
len(list(model.wv.vocab.keys())) # Men, med min_count = 10 har du kun 1650 obs... Flere tweets mere text.

3174

In [23]:
check_model(model)

Most similar to president:

('obama', 0.7636547684669495)
('bush', 0.7594131827354431)
('press', 0.7548365592956543)
('wyden', 0.7422565817832947)
('cupcake', 0.7402195930480957)
('dr.', 0.7389487028121948)
('camera', 0.7380211353302002)
('mac', 0.7356566190719604)
('kkk', 0.7342849969863892)
('\\ni', 0.7300539016723633)


Most common words:
you that n't


Least common words:
dode blair tannerite


Sanity sim check1:
0.30876151574421223 0.1499693290056254


Sanity sim check2:
man


### Political top comments

In [24]:
spec_path = path + "top_comments_politics.pkl"


pkl_file = open(spec_path, 'rb') # from feature set.
comments5 = pickle.load(pkl_file)
pkl_file.close()

# tweets are back from 140 character limit; not 240!
sentences = [prep2(sent) for sent in nltk.sent_tokenize(pd.Series(comments5).to_string())]
model = gensim.models.Word2Vec(sentences, iter=10, min_count=10, size=300,) # virker meget bedre med min_count = 10
len(list(model.wv.vocab.keys())) # Men, med min_count = 10 har du kun 1650 obs... Flere tweets mere text.

10107

In [25]:
check_model(model)

Most similar to president:

('presi', 0.6865994930267334)
('pres', 0.6773925423622131)
('potus', 0.6593327522277832)
('rogers', 0.46169722080230713)
('arpaio', 0.420728862285614)
('mrs.', 0.40820616483688354)
('pardon', 0.40549448132514954)
('nation', 0.40211716294288635)
('presidential', 0.39559489488601685)
('donald', 0.38100704550743103)


Most common words:
that this you


Least common words:
thermostat longmont equity


Sanity sim check1:
0.3178115025292681 0.026592939916269723


Sanity sim check2:
man


# News top comment

In [26]:
spec_path = path + "top_comments_news.pkl"


pkl_file = open(spec_path, 'rb') # from feature set.
comments6 = pickle.load(pkl_file)
pkl_file.close()

# tweets are back from 140 character limit; not 240!
sentences = [prep2(sent) for sent in nltk.sent_tokenize(pd.Series(comments6).to_string())]
model = gensim.models.Word2Vec(sentences, iter=10, min_count=10, size=300,) # virker meget bedre med min_count = 10
len(list(model.wv.vocab.keys())) # Men, med min_count = 10 har du kun 1650 obs... Flere tweets mere text.

10723

In [27]:
check_model(model)

Most similar to president:

('donald', 0.7159775495529175)
('erdogan', 0.683106005191803)
('election', 0.6680687665939331)
('obama', 0.6522588133811951)
('trump', 0.6491758227348328)
('flynn', 0.6461325287818909)
('administration', 0.6432797312736511)
('mueller', 0.6407721042633057)
('candidate', 0.6254674196243286)
('comey', 0.6114505529403687)


Most common words:
that you this


Least common words:
psu catman wham


Sanity sim check1:
0.2599127533802413 0.07999153167409581


Sanity sim check2:
man


## Aggregated Reddit:

In [28]:
comments_all = comments1 + comments2 + comments3 + comments4 + comments5 + comments6


# tweets are back from 140 character limit; not 240!
sentences = [prep2(sent) for sent in nltk.sent_tokenize(pd.Series(comments_all).to_string())]
model = gensim.models.Word2Vec(sentences, iter=10, min_count=10, size=300,) # virker meget bedre med min_count = 10
len(list(model.wv.vocab.keys())) # Men, med min_count = 10 har du kun 1650 obs... Flere tweets mere text.

20130

In [29]:
check_model(model)

Most similar to president:

('presi', 0.6867668628692627)
('pres', 0.6520718336105347)
('versa', 0.6129910945892334)
('potus', 0.5942607522010803)
('presidential', 0.4036596417427063)
('trump', 0.40084153413772583)
('appointee', 0.3959399461746216)
('impeached', 0.39468443393707275)
('elizondo', 0.39404264092445374)
('erdogan', 0.39086031913757324)


Most common words:
that you this


Least common words:
psu catman wham


Sanity sim check1:
0.2835504692155338 0.04269146825776404


Sanity sim check2:
man


In [37]:
2+2

4

# Full Aggrigate

In [30]:
all_text = comments_all + list(df_unlabled_big1.text) + list(df_unlabled_big2.text) + list(df_unlabled.text)

In [31]:
len(all_text)

2539464

In [32]:
sentences = [prep2(sent) for sent in nltk.sent_tokenize(pd.Series(all_text).to_string())]

In [33]:
# this time we set a bit more options:

# Set values for various parameters
feature_size = 300    # Word vector dimensionality  
window_context = 30          # Context window size                                                                                    
min_word_count = 10   # Minimum word count                        
sample = 1e-3   # Downsample setting for frequent words


# tweets are back from 140 character limit; not 240!
model = gensim.models.Word2Vec(sentences, size=feature_size, window=window_context, min_count=min_word_count, sample=sample, iter=50)
                               
len(list(model.wv.vocab.keys())) # Men, med min_count = 10 har du kun 1650 obs... Flere tweets mere text.

38157

In [34]:
check_model(model, "president")

Most similar to president:

('administration', 0.6124956607818604)
('pres', 0.6078202128410339)
('potus', 0.48590153455734253)
('admin', 0.4723565876483917)
('presi', 0.3922380805015564)
('donald', 0.37990128993988037)
('president-elect', 0.3515026569366455)
('trump', 0.3480735421180725)
('realdonaldtrump', 0.32141876220703125)
('versa', 0.3123326301574707)


Most common words:
you that this


Least common words:
peterwsj bkesling opportunityagenda


Sanity sim check1:
0.0604121496896793 -0.0734339004365574


Sanity sim check2:
man


# Pickling model:

In [38]:
output = open('word2vec_model_back_up.pkl', 'wb')
pickle.dump(model, output)
output.close()

In [39]:
model.wv.word_vec("potus").shape

(300,)